# 数据载入与存储

本实验中，我们将介绍数据的读取与输出

本实验使用的包及版本如下

In [ ]:
# !pip install numpy==1.16.2
# !pip install pandas==0.24.2
# !pip install matplotlib==3.0.3

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345) # 设置伪随机数生成器的种子起点，保证实验结果一致
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6)) # 格式设置
np.set_printoptions(precision=4, suppress=True) # 格式设置

## 文本格式的数据读写

Pandas可以读取各种格式的数据

输入上述代码，按tab键补全，可以查看所有函数

In [ ]:
# pd.read_

由于现实中的数据非常混乱，一些数据加载函数的可选参数非常复杂

### !type

查看原始内容，书上Unix shell使用的是!cat

windows中使用!type

注意这里斜杠的方向，与Unix shell的方向不同

In [3]:
!type examples\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


可以看到这个文件是逗号分隔的，下面我们使用read_csv将该文件读入DataFrame

### read_csv

逗号是默认分隔符

In [4]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### read_table

read_table，允许自己指定分隔符

In [5]:
# 这里用sep指定','为默认分隔符
pd.read_table('examples/ex1.csv', sep=',')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### header参数

有的文件没有头行，比如下面这个

In [6]:
!type examples\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


设置header=None，会默认给个数字当表头

In [7]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 参数names

或者，用names=自己设置表头

In [8]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 参数index_col

如果你想指定第四列为索引，可以使用，index_col=

In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


想要设置分层所以

如打类分成两个个one two

one下分四个abcd

two下分四个abcd

我们看下面这个文件

In [10]:
!type examples\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


 则使用index_col指定两列索引['key1', 'key2']

In [11]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

#### 使用正则表达式做分隔符

有的时候，一张表的分隔符不是固定的，考虑下面这种情况

In [12]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

该文档是数量不确定的空格分隔，指定分隔符为正则表达式\s+

In [13]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


本例中，由于列名的数量少一个，read_table会自动推断第一列为索引

#### skiprows

解析异常，如下面这个文档

In [14]:
!type examples\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


这里面有一些注释行
 
使用skiprows=[0, 2, 3]跳过

In [15]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 处理缺失值

通常pandas中用NULL或NA来标记

In [16]:
!type examples\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [26]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


判断是否为缺失值

In [27]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


##### na_values

可以传入一个类表或者一组字符串来指定缺失值

In [28]:
result = pd.read_csv('examples/ex5.csv', na_values=['Null'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


每列可以指定不同的缺失值表示

In [24]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 6.1.1 分块读入文本文件（Reading Text Files in Pieces）

#### 显示调整

对pandas的显示进行调整，只显示十列

In [29]:
pd.options.display.max_rows = 10

In [44]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


#### 指定行数

只读出前五行

In [45]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


chunker就是分块的意思

chunksize=1000，指定每一块的行数

In [46]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

比如，0-999是第一块，1000-1999是第二块儿

返回值为TextParser对象，允许遍历

In [47]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
for piece in chunker:
    print(piece)

          one       two     three      four key
0    0.467976 -0.038649 -0.295344 -1.824726   L
1   -0.358893  1.404453  0.704965 -0.200638   B
2   -0.501840  0.659254 -0.421691 -0.057688   G
3    0.204886  1.074134  1.388361 -0.982404   R
4    0.354628 -0.133116  0.283763 -0.837063   Q
..        ...       ...       ...       ...  ..
995  2.311896 -0.417070 -1.409599 -0.515821   M
996 -0.479893 -0.650419  0.745152 -0.646038   H
997  0.523331  0.787112  0.486066  1.093156   D
998 -0.362559  0.598894 -1.843201  0.887292   W
999 -0.096376 -1.012999 -0.657431 -0.573315   K

[1000 rows x 5 columns]
           one       two     three      four key
1000  0.467976 -0.038649 -0.295344 -1.824726   T
1001 -0.358893  1.404453  0.704965 -0.200638   J
1002 -0.501840  0.659254 -0.421691 -0.057688   R
1003  0.204886  1.074134  1.388361 -0.982404   S
1004  0.354628 -0.133116  0.283763 -0.837063   B
...        ...       ...       ...       ...  ..
1995  2.311896 -0.417070 -1.409599 -0.515821   L
1996 -0

In [48]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

for piece in chunker:
    print(piece['key'].value_counts())

S    48
O    44
F    40
Q    39
J    39
     ..
1    13
9    11
2    11
0     9
5     9
Name: key, Length: 36, dtype: int64
O    48
L    44
X    40
I    39
R    38
     ..
3    16
1    16
6    14
2    14
7    12
Name: key, Length: 36, dtype: int64
A    40
O    40
E    39
X    39
M    38
     ..
2    14
4    14
5    14
8    14
1    11
Name: key, Length: 36, dtype: int64
X    43
J    41
V    38
Q    38
D    38
     ..
9    15
0    15
8    13
4    12
7    12
Name: key, Length: 36, dtype: int64
E    54
Q    42
L    40
H    39
K    38
     ..
5    17
8    16
7    13
9     8
1     8
Name: key, Length: 36, dtype: int64
Y    42
K    41
F    41
X    38
V    37
     ..
6    17
3    16
4    13
2    12
0     8
Name: key, Length: 36, dtype: int64
C    41
L    41
W    39
Z    39
X    38
     ..
2    17
1    16
5    15
3    11
6     8
Name: key, Length: 36, dtype: int64
J    42
M    40
D    39
X    39
E    39
     ..
7    14
3    13
5    13
1    12
9     5
Name: key, Length: 36, dtype: int64
O    42


In [41]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [49]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [50]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### 6.1.2 将数据写入文本格式（Writing Data to Text Format）

In [57]:
import numpy as np
import pandas as pd
np.random.seed(12345) # 设置伪随机数生成器的种子起点，保证实验结果一致
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6)) # ？
np.set_printoptions(precision=4, suppress=True) # ？

读出文本，读出的变量存在data里

In [58]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [ ]:
# data.to_

写入文档

In [59]:
data.to_csv('examples/out.csv')

查看写入的文档

In [62]:
!type examples\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


sys.stdout输出到屏幕，但是，使用|分割

In [63]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值使用NULL填充缺失值

In [64]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


写入文档，去掉index，去掉header

In [65]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


写入文档，列名为a,b,c

In [66]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


生成时间数据，从2000年1月1日开始，期限为7天

In [67]:
dates = pd.date_range('1/1/2000', periods=7)

可以看到这里的数据类型是datetime

In [68]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

生成0-6七个数字，索引为刚才生成的日期

In [69]:
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

将文档写入文件

In [70]:
ts.to_csv('examples/tseries.csv')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


查看刚才保存的文档

In [71]:
!type examples\tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 6.1.3 处理分隔格式 Working with Delimited Formats

有些情况read_csv解决不了

In [72]:
!type examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


对于这种带双引号的文档，首先导入csv包，读出csv文件

In [73]:
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

读出来的reader是一个可迭代对象

In [74]:
reader

需要使用for循环读出来

In [75]:
for line in reader:
    print(line)
    print(type(line))

['a', 'b', 'c']
<class 'list'>
['1', '2', '3']
<class 'list'>
['1', '2', '3']
<class 'list'>


可以看到是类似列表的东西

将文件读取为行的列表

In [78]:
with open('examples/ex7.csv') as f:
    lines = tuple(csv.reader(f))
lines

(['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3'])

哦~，所以这个可迭代对象，就缺个马甲是嘛

In [79]:
with open('examples/ex7.csv') as f:
    lines = tuple(csv.reader(f))
lines

(['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3'])

哈哈哈似乎领悟到了什么真谛

In [82]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
header, values = lines[0], lines[1:]
header

['a', 'b', 'c']

In [83]:
values

[['1', '2', '3'], ['1', '2', '3']]

哦~再拆成字典格式，好好学学！！

In [84]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

别问，问就是背下来

### 6.1.4 JSON Data

一回生二回熟，JavaScript Object Notation

常用的标准格式，比csv更为自由

看下面这个例子

In [1]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

我们使用Python中内置的json标准库读取json文件

json字符串转换为python形式

In [3]:
import json
import pandas as pd
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

Python形式，转换回json

In [5]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

可以自由转换数据结构

如result变量里面的‘siblings’对应的字典

字典里面取出name和age作为列

In [6]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


哇哦~还有这种操

下面这个json文件

In [7]:
!type examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


pd.read_json可以直接报json数据集按照指定的次序转换为Series或者DataFrame

In [8]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


在转换为json

In [9]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


注意跟上面不同的保存方式

In [10]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### 6.1.5 网络抓取 XML and HTML: Web Scraping

html和xml格式文件的读取

In [ ]:
# !pip install lxml

In [ ]:
# !pip install beautifulsoup4 html5lib

美国FDIC政府机构下载的显示银行倒闭数据的HTML文件

In [11]:
!type examples\fdic_failed_bank_list.html

<!DOCTYPE html>
<!-- saved from url=(0057)https://www.fdic.gov/bank/individual/failed/banklist.html -->
<html lang="en-US"><!-- Content language is American English. --><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>FDIC: Failed Bank List</title>
<!-- Meta Tags -->
<meta charset="UTF-8">
<!-- Unicode character encoding -->
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<!-- Turns off IE Compatiblity Mode -->
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
<!-- Makes it so phones don't auto zoom out. -->
<meta name="author" content="DRR">
<meta http-equiv="keywords" name="keywords" content="banks, financial institutions, failed, failure, closing, deposits, depositors, banking services, assuming institution, acquiring institution, claims">
<!-- CSS -->
<link rel="stylesheet" type="text/css" href="./fdic_failed_bank_list_files/responsive.css">
<link rel="stylesheet" type="text/css" href="./fdic_failed_bank_list

			  <td class="closing">February 11, 2011</td>
			  <td class="updated">January 22, 2013</td>
			  </tr><tr class="even">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/sunshinestate.html">Sunshine State Community Bank</a></td>
			  <td class="city">Port Orange</td>
			  <td class="state">FL</td>
			  <td class="cert">35478</td>
			  <td class="ai">Premier American Bank, N.A.</td>
			  <td class="closing">February 11, 2011</td>
			  <td class="updated">August 8, 2016</td>
			  </tr><tr class="odd">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/commfirst_il.html">Community First Bank Chicago</a></td>
			  <td class="city">Chicago</td>
			  <td class="state">IL</td>
			  <td class="cert">57948</td>
			  <td class="ai">Northbrook Bank &amp; Trust Company</td>
			  <td class="closing">February 4, 2011</td>
			  <td class="updated">August 20, 2012</td>
			  </tr><tr class="even">
			  <td class="institution"><a href="ht

			  <td class="state">GA</td>
			  <td class="cert">34301</td>
			  <td class="ai">United Bank</td>
			  <td class="closing">December 5, 2008</td>
			  <td class="updated">August 16, 2012</td>
			  </tr><tr class="odd">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/pff.html">PFF Bank &amp; Trust</a></td>
			  <td class="city">Pomona</td>
			  <td class="state">CA</td>
			  <td class="cert">28344</td>
			  <td class="ai">U.S. Bank, N.A.</td>
			  <td class="closing">November 21, 2008</td>
			  <td class="updated">January 4, 2013</td>
			  </tr><tr class="even">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/downey.html">Downey Savings &amp; Loan</a></td>
			  <td class="city">Newport Beach</td>
			  <td class="state">CA</td>
			  <td class="cert">30968</td>
			  <td class="ai">U.S. Bank, N.A.</td>
			  <td class="closing">November 21, 2008</td>
			  <td class="updated">January 4, 2013</td>
			  </tr><tr class="odd">

pd.read_html有很多选项，但默认的情况下，会搜索并尝试解析所有包含在<table>标签中的表格型数据，返回的结果是DataFrame对象的列表

In [12]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


可以进行一些简单的清理工作，比如计算每年银行倒闭的数量

In [13]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
5     2015-10-02
6     2015-10-02
7     2015-07-10
8     2015-05-08
9     2015-02-27
10    2015-02-13
11    2015-01-23
12    2015-01-16
13    2014-12-19
14    2014-11-07
15    2014-10-24
16    2014-10-17
17    2014-07-25
18    2014-07-18
19    2014-06-27
20    2014-06-20
21    2014-06-20
22    2014-05-30
23    2014-05-23
24    2014-05-16
25    2014-04-25
26    2014-02-28
27    2014-02-28
28    2014-01-31
29    2014-01-24
         ...    
517   2008-05-09
518   2008-03-07
519   2008-01-25
520   2007-10-04
521   2007-09-28
522   2007-02-02
523   2004-06-25
524   2004-03-19
525   2004-03-12
526   2004-02-14
527   2003-11-14
528   2003-05-09
529   2003-02-07
530   2002-12-17
531   2002-11-08
532   2002-09-30
533   2002-06-27
534   2002-06-26
535   2002-03-28
536   2002-03-01
537   2002-02-07
538   2002-02-01
539   2002-01-18
540   2002-01-11
541   2001-09-07
542   2001-07-27
543   2001-05-03
544   2001-02-

??没太看懂这行，dt是是啥，year是啥？

例
 
 s.dt.hour

 s.dt.second

 s.dt.quarter
 
 dt.year,获得时间中的年份，value_counts()并统计这一列中有多少个

In [15]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

#### 6.1.5.1 解析XML使用lxml.objectify

XML,是一种常用的结构化数据格式，使用元数据支持分层、嵌套数据

下面是纽约大都会交通局发布的一份关于公交、火车服务的数据集

```XML
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
```

使用lxml.objectify

In [17]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot() # 获得对XML文件的根节点的引用

返回一个迭代器，可以产生每一个<INDICATOR>XML元素

In [18]:
root

<Element PERFORMANCE at 0x1ae3e848dc8>

In [20]:
data = []
# 储存所有数据的列表

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']
# 这几个标签下的值不要

for elt in root.INDICATOR:
    # 一组数据
    el_data = {}
    # 储存该组数据的字典
    for child in elt.getchildren():
        # 取出一列数据
        if child.tag in skip_fields:
            # 如果这列的标签，在我们上面定义的列表里
            continue
            # 舍弃，继续
        el_data[child.tag] = child.pyval
        # 如果不在，标签作为key值，这列的pyval作为value值加入字典
    data.append(el_data)
    # 该组数据加入总体数据列表

In [21]:
len(data)

648

In [22]:
data[1]

{'AGENCY_NAME': 'Metro-North Railroad',
 'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
 'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
 'PERIOD_YEAR': 2008,
 'PERIOD_MONTH': 2,
 'CATEGORY': 'Service Indicators',
 'FREQUENCY': 'M',
 'INDICATOR_UNIT': '%',
 'YTD_TARGET': 95.0,
 'YTD_ACTUAL': 96.0,
 'MONTHLY_TARGET': 95.0,
 'MONTHLY_ACTUAL': 95.0}

可以看到生成的data是一个类表，每个元素是一个字典，所有字典的key值顺序对应

最将包含字典的列表转换为DataFrame

In [23]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


XML数据如果更复杂，比如每个标签包含元数据

好吧没懂。。

In [24]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [25]:
root
root.get('href')
root.text

'Google'

## 6.2 二进制格式 Binary Data Formats

In [26]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


保存为二进制格式

In [27]:
frame.to_pickle('examples/frame_pickle')

In [28]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pickle只能作为短期的存储格式，难以保证长期有效

### 6.2.1 使用HDF5格式

用于存储大量的科学数组数据

In [50]:
import numpy as np
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [51]:
store['obj1_col']

0     0.309250
1     0.680037
2     0.989194
3    -1.362328
4    -0.533243
5    -1.187825
6    -1.029347
7    -1.077026
8     0.933109
9    -1.806425
10   -1.347547
11    0.592021
12    0.178162
13   -1.090297
14    0.852064
15    0.280577
16    0.671429
17   -0.167449
18    0.184109
19   -0.400377
20   -0.946493
21   -1.740762
22   -1.744523
23    0.287905
24   -0.735220
25    1.071231
26   -0.828809
27   -0.094955
28    0.409779
29    0.719787
        ...   
70    1.831866
71    1.571385
72    0.439236
73    0.603217
74   -0.300065
75   -1.053181
76   -0.168029
77    1.839252
78    0.413853
79    0.931166
80    0.106162
81    0.208946
82   -0.725353
83   -1.010852
84   -1.026712
85    2.229865
86   -0.738917
87    0.017304
88   -0.873912
89    0.504836
90   -0.099655
91   -0.608003
92   -1.631771
93    0.567855
94   -0.060269
95    0.993129
96   -0.592249
97   -0.784817
98   -0.325682
99    0.360261
Name: a, Length: 100, dtype: float64

In [52]:
store.put('obj2', frame, format='table')

In [53]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-1.347547
11,0.592021
12,0.178162
13,-1.090297
14,0.852064
15,0.280577


In [54]:
store.close()

In [56]:
# frame.to_hdf('mydata.h5', 'obj3', format='table')
# pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

In [58]:
# import os
# os.remove('mydata.h5')

### 6.2.2 Reading Microsoft Excel Files

In [59]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [60]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [61]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [62]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [63]:
frame.to_excel('examples/ex2.xlsx')

In [65]:
!type examples/ex2.xlsx

命令语法不正确。


## 6.3 Interacting with Web APIs

获取github上最新的30条关于pandas的问题，发送一个HTTP GET请求

In [66]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

Response对象的json方法将返回一个包含解析为本地Python对象的JSON字典

In [67]:
data = resp.json()
data[0]['title']

'df.Truncate now fails when arguments have different tzinfo'

In [68]:
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/29856',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/29856/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/29856/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/29856/events',
  'html_url': 'https://github.com/pandas-dev/pandas/issues/29856',
  'id': 528629989,
  'node_id': 'MDU6SXNzdWU1Mjg2Mjk5ODk=',
  'number': 29856,
  'title': 'df.Truncate now fails when arguments have different tzinfo',
  'user': {'login': 'JrtPec',
   'id': 4570408,
   'node_id': 'MDQ6VXNlcjQ1NzA0MDg=',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/4570408?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/JrtPec',
   'html_url': 'https://github.com/JrtPec',
   'followers_url': 'https://api.github.com/users/JrtPec/followers',
   'following_url': 'https:

data中的每个元素都是一个包含GitHub问题页面上所有数据的字典

我们可以将data直接传给DataFrame，并提取感兴趣的字段

In [69]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,29856,df.Truncate now fails when arguments have diff...,[],open
1,29855,ValueError when slicing dataframe with duplica...,[],open
2,29854,CI: styled excel failure in the doc build,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
3,29853,WIP: implement scalar ops blockwise,[],open
4,29852,TST: Fix locale test that fails in GitHub actions,"[{'id': 42670965, 'node_id': 'MDU6TGFiZWw0MjY3...",open
5,29851,STY: 'open()' and 'close()' to 'with' context ...,[],open
6,29850,TYP: Typing annotations in clipboard,[],open
7,29849,add to_cyclical function for cyclical data #en...,[],open
8,29848,MAINT: Fix grammar in user_guide/scale.rst,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
9,29847,PERF: perform reductions block-wise,[],open


## 6.4 与数据库交互 Interacting with Databases

通常，企业中的数据不是存储在文本或Excel文件中的

而是基于SQL的关系型数据库

SQL Server，PostgreSQL，MySQL等


特别鸣谢！！！群里的小伙伴

@black$dust

@

- 数据库连接
- 执行sql语句
- 提交
- 关闭连接

#### sqlite3

使用sqlite3创建数据库的连接

In [70]:
import sqlite3

####  数据库连接
#### sql命令

在Python中用三引号创建一个多行文本变量

该变量为sql命令

用于创建一个名字叫test的表格

里面有4列数据

In [73]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

其中a和b字段是字符串类型，最长20

c是十进制8位存储

d是整数类型

#### 打开或创建数据库文件

当我们指定的数据库文件不存在的时候

连接对象会自动创建数据库文件

如果数据库文件已经存在，直接打开该数据库文件

In [76]:
con = sqlite3.connect('mydata_1.sqlite')

其中.sqlite，表面该文件的类型是文件型数据库

####  执行命令

In [77]:
con.execute(query)

####  提交命令

对数据库产生修改的语句，要commit才会生效

如果是查询，就只要execute提交语句。

In [78]:
con.commit()

####  执行多条命令并提交

将数据一行一行插进去

In [79]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

#### 读出所有数据

```
select * from test
```

*是通配符，类似正则表达式，在test表格里读出所有数据

In [80]:
cursor = con.execute('select * from test')
cursor

返回的这个东西类似一个指针，指向数据表中某一行，以此类推的话就一定是个可迭代对象

使用fetchall()函数读出 

In [81]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

这里读出类表格内容，但是没有读出列名

列名在cursor的description属性中，所以，我们可以将它读出

In [82]:
for x in cursor.description:
    print(x[0]) 

a
b
c
d


将列名作为dataFrame的columns，row作为每行的值，创建DataFrame

In [83]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [84]:
con.close()

#### sqlalchemy

也是一个库，比较简便

我尽力了，以后再说

In [85]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


In [ ]:
# linux系统删除命令
# !rm mydata.sqlite

In [87]:
# windows系统删除命令
!del mydata_1.sqlite

 ## 其他

[SQL 语法](https://www.w3school.com.cn/sql/sql_syntax.asp)